In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.random.seed(601)

import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [37]:
# 날짜 처리
data = pd.read_csv('data/201901-202003.csv')

In [38]:
data.drop(['CARD_CCG_NM', 'HOM_CCG_NM'   , 'AGE', 'SEX_CTGO_CD', 'FLC','CSTMR_CNT'], axis=1, inplace=True)

In [39]:
data

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AMT,CNT
0,201901,강원,건강보조식품 소매업,강원,311200,4
1,201901,강원,건강보조식품 소매업,강원,1374500,8
2,201901,강원,건강보조식품 소매업,강원,818700,6
3,201901,강원,건강보조식품 소매업,강원,1717000,5
4,201901,강원,건강보조식품 소매업,강원,1047300,3
...,...,...,...,...,...,...
24697787,202003,충북,휴양콘도 운영업,충북,43300,4
24697788,202003,충북,휴양콘도 운영업,충북,35000,3
24697789,202003,충북,휴양콘도 운영업,충북,188000,6
24697790,202003,충북,휴양콘도 운영업,충북,99000,6


In [32]:
data[pd.isnull(data)].sum()

REG_YYMM        0.0
CARD_SIDO_NM    0.0
STD_CLSS_NM     0.0
HOM_SIDO_NM     0.0
AMT             0.0
CNT             0.0
dtype: float64

In [ ]:
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [ ]:
data

In [5]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'   , 'AGE', 'SEX_CTGO_CD', 'FLC'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'year', 'month']


In [40]:
df = data.copy()
columns = ['REG_YYMM','CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM']
df = df.groupby(columns).sum().reset_index(drop=False)

In [41]:
df

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AMT,CNT
0,201901,강원,건강보조식품 소매업,강원,147831746,1105
1,201901,강원,건강보조식품 소매업,광주,127000,5
2,201901,강원,건강보조식품 소매업,대구,155000,7
3,201901,강원,건강보조식품 소매업,서울,151000,6
4,201901,강원,건강보조식품 소매업,전북,90000,5
...,...,...,...,...,...,...
106381,202003,충북,휴양콘도 운영업,경기,2170050,20
106382,202003,충북,휴양콘도 운영업,서울,6830430,82
106383,202003,충북,휴양콘도 운영업,세종,354000,3
106384,202003,충북,휴양콘도 운영업,인천,88250,5


In [42]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [ ]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [43]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
x = train_num.drop(['AMT', 'CNT'], axis=1)
y = np.log1p(train_num['AMT'])

In [44]:
k = int(len(x)*0.9)

In [45]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [46]:
import lightgbm as lgb

In [47]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [48]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [49]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 2.04429
[200]	valid_0's rmse: 1.7275
[300]	valid_0's rmse: 1.55046
[400]	valid_0's rmse: 1.43544
[500]	valid_0's rmse: 1.34365
[600]	valid_0's rmse: 1.27002
[700]	valid_0's rmse: 1.21901
[800]	valid_0's rmse: 1.17286
[900]	valid_0's rmse: 1.13219
[1000]	valid_0's rmse: 1.09946
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 1.09946


In [50]:
x

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM
68516,201910,7,39,10
99177,202002,13,37,0
58507,201909,1,17,7
18585,201903,10,20,11
29995,201905,2,37,8
...,...,...,...,...
21243,201903,16,25,15
45891,201907,6,18,0
42613,201906,15,23,11
43567,201907,0,32,6


In [64]:
# 예측 템플릿 만들기
REG_YYMMs     = [202004, 202007]
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
#AGEs          = df_num['AGE'].unique()
#SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
#FLCs          = df_num['FLC'].unique()
#years         = [2020]
#months        = [4, 7]

temp = []
for REG_YYMM in REG_YYMMs:
    for CARD_SIDO_NM in CARD_SIDO_NMs:
        for STD_CLSS_NM in STD_CLSS_NMs:
            for HOM_SIDO_NM in HOM_SIDO_NMs:
                #for AGE in AGEs:
                    #for SEX_CTGO_CD in SEX_CTGO_CDs:
                        #for FLC in FLCs:
                            #for year in years:
                                #for month in months:
                temp.append([REG_YYMM, CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM]) # , AGE, SEX_CTGO_CD, FLC, , year, month
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=x.columns)

In [65]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM
0,202004,0,0,0
1,202004,0,0,4
2,202004,0,0,5
3,202004,0,0,8
4,202004,0,0,13
...,...,...,...,...
23693,202007,16,30,11
23694,202007,16,30,12
23695,202007,16,30,14
23696,202007,16,30,15


In [66]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
#temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [67]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,0,0,1.103752e+08
1,202004,0,1,9.179540e+08
2,202004,0,2,3.642660e+08
3,202004,0,3,2.873264e+07
4,202004,0,4,1.377231e+06
...,...,...,...,...
1389,202007,16,36,2.958295e+08
1390,202007,16,37,1.612656e+10
1391,202007,16,38,2.249548e+07
1392,202007,16,39,1.029161e+08


In [68]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [69]:
temp

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,1.103752e+08
1,202004,강원,골프장 운영업,9.179540e+08
2,202004,강원,과실 및 채소 소매업,3.642660e+08
3,202004,강원,관광 민예품 및 선물용품 소매업,2.873264e+07
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.377231e+06
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,2.958295e+08
1390,202007,충북,한식 음식점업,1.612656e+10
1391,202007,충북,호텔업,2.249548e+07
1392,202007,충북,화장품 및 방향제 소매업,1.029161e+08


In [70]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('20200706_3.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,110375155.0
1,202004,강원,골프장 운영업,917954020.0
2,202004,강원,과실 및 채소 소매업,364265979.0
3,202004,강원,관광 민예품 및 선물용품 소매업,28732635.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1377231.0
